# Import

In [ ]:
%pip install seaborn scikit-learn scipy

In [ ]:
import os
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score, davies_bouldin_score

from sklearn.manifold import TSNE, Isomap
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import linkage, dendrogram

from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder


sns.set_style("darkgrid")

# Load/Split

Skup podataka: https://archive.ics.uci.edu/dataset/374/appliances+energy+prediction

In [ ]:
file_path = './ObesityDataSet.csv'
df = pd.read_csv(file_path, sep=',')

# Priprema i analiza podataka

## Opis podataka

- $Gender$ - pol
- $Age$ - starost
- $Height$ - visina
- $Weight$ - kilaža
- $family\_history\_with\_overweight$ - da li su članovi porodice gojazni (yes-no)
- $FAVC$ - čest unos visoko kalorične hrane
- $FCVC$ - učestalost unosa povrća
- $NCP$ - broj glavnih jela
- $CAEC$ - unos hrane između obroka - 4 jedinstvene vrednosti
- $SMOKE$ - pušač (da-ne)
- $CH2O$ - unos vode u toku dana
- $SCC$ - da li vode računa o unosu kalorija (da-ne)
- $FAF$ - učestalost fizičke aktivnosti
- $TUE$ - vreme korišćenja tehnologije
- $CALC$ - unos alkohola - 4 jedinstvene vrednosti
- $MTRANS$ - korišćenje prevoznih sredstava - 5 jedinstvene vrednosti
- $NObeyesdad$ - Nivo gojaznosti
    - $Underweight$
    - $Overweight$
    - $Obesity I$
    - $Obesity II$
    - $Obesity III$

In [ ]:
print(f"Broj ulaza u skupu podataka: {len(df)}")

In [ ]:
print(f"Broj ulaznih promenljivih u skupu podataka: {len(df.iloc[0])}")

- Ukupno imamo <span style="color:#03d3fc">17 ulazne promenljive</span> i <span style="color:#fc036f">1 ciljnu promenljivu</span>
- Ukupno imamo <span style="color:#03d3fc">1688 ulaza u train skupu podataka</span>
- Koristimo 80-20 podelu za trening i test skup

In [ ]:
df.info()

- Nema nedostajućih vrednosti u skupu
- $Gender$, $family\_history\_with\_overweight$, $FAVC$, $CAEC$, $SMOKE$, $SCC$, $CALC$, $MTRANS$ i $NObeyesdad$ su object

## Obrada atributa

In [ ]:
df['Gender'] = df['Gender'].map({'Female': 0, 'Male': 1})
df['family_history_with_overweight'] = df['family_history_with_overweight'].map({'yes': 1, 'no': 0})
df['FAVC'] = df['FAVC'].map({'yes': 1, 'no': 0})
df['SMOKE'] = df['SMOKE'].map({'yes': 1, 'no': 0})
df['SCC'] = df['SCC'].map({'yes': 1, 'no': 0})

In [ ]:
df = pd.get_dummies(df, columns=['CAEC'], prefix='CAEC',dtype=int)
df = pd.get_dummies(df, columns=['CALC'], prefix='CALC',dtype=int)
df = pd.get_dummies(df, columns=['MTRANS'], prefix='MTRANS',dtype=int)

In [ ]:
label_encoder = LabelEncoder()

df['NObeyesdad'] = label_encoder.fit_transform(df['NObeyesdad'])

In [ ]:
df.info()

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

## Raspodele podataka

In [ ]:
def visualize_attribute(train: pd.DataFrame, attribute: str) -> None:
    plt.figure(facecolor='darkblue')
    color = np.random.rand(3,)
    _, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    sns.histplot(data=train, x=attribute, kde=False, ax=axes[0, 0], color=color)
    axes[0, 0].set_title('Histogram')
    
    sns.boxplot(data=train, x=attribute, ax=axes[0, 1], color=color)
    axes[0, 1].set_title('Boxplot')
    
    sns.violinplot(data=train, x=attribute, ax=axes[1, 0], color=color)
    axes[1, 0].set_title('Violin Plot')
    
    sns.scatterplot(data=train, x=attribute, y='NObeyesdad', ax=axes[1, 1], color=color, alpha=0.2)
    axes[1, 1].set_title('Scatter Plot')
    
    plt.show()

attributes = list(train.keys())
for attribute in attributes:
    visualize_attribute(train, attribute)

## Normalizacija atributa

In [ ]:
def transform_skewed_attributes(train, transf_map:dict, skew_threshold=1.0):
    transformer = PowerTransformer(method='yeo-johnson')
    train_cp = train.copy()
    for column in train_cp.columns:
        if train_cp[column].dtype in [np.float64, np.int64]:
            skewness = train_cp[column].skew()
            if abs(skewness) > skew_threshold:
                transformer = PowerTransformer(method='yeo-johnson')
                transformer = transformer.fit(train_cp[[column]])
                train_cp[column] = transformer.transform(train_cp[[column]])
                if column in transf_map.keys():
                    transf_map[column].append(transformer)
                else:
                    transf_map[column] = [transformer]

    return train_cp

def scale(train, transf_map:dict):
    train_cp = train.copy()
    for column in train.columns:
        if train_cp[column].dtype in [np.float64, np.int64]:
            scaler = MinMaxScaler(feature_range=(0, 1))
            scaler = scaler.fit(train_cp[[column]])
            train_cp[column] = scaler.transform(train_cp[[column]])
            if column in transf_map.keys():
                    transf_map[column].append(scaler)
            else:
                transf_map[column] = [scaler]
    return train_cp

def apply_transf(X, transf_map:dict):
    result = X.copy()
    for attribute in transf_map.keys():
        for tr in transf_map[attribute]:
            result[attribute] = tr.transform(result[[attribute]])
    return result

## Vizuelizacija cluster-a

In [ ]:
def visualize_clusters(data, clusterer, perplexity=30, n_iter=1000):
    if hasattr(clusterer,'predict'):
        labels = clusterer.predict(data)
    else:
        labels = clusterer.fit_predict(data)
    
    tsne = TSNE(n_components=2, perplexity=perplexity, n_iter=n_iter, random_state=42)
    tsne_results = tsne.fit_transform(data)
    
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=labels, cmap='viridis', alpha=0.6)
    plt.legend(*scatter.legend_elements(), title="Klasteri")
    plt.title(f'Vizuelizacija klastera')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()

def evaluate_kmeans(data, min_clusters=5, max_clusters=15):
    silhouette_scores = []
    davies_bouldin_scores = []
    cluster_range = range(min_clusters, max_clusters + 1)
    
    results = {}
    for n_clusters in cluster_range:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        labels = kmeans.fit_predict(data)
        
        silhouette_avg = silhouette_score(data, labels)
        davies_bouldin_avg = davies_bouldin_score(data, labels)
        
        silhouette_scores.append(silhouette_avg)
        davies_bouldin_scores.append(davies_bouldin_avg)
        
        results[n_clusters] = [kmeans, silhouette_avg, davies_bouldin_avg]
    
    # Plotting the scores
    fig, ax1 = plt.subplots()

    ax1.set_xlabel('Number of clusters')
    ax1.set_ylabel('Silhouette Score', color='tab:blue')
    ax1.plot(cluster_range, silhouette_scores, 'o-', color='tab:blue', label='Silhouette Score')
    ax1.tick_params(axis='y', labelcolor='tab:blue')
    ax1.legend(loc='upper left')

    ax2 = ax1.twinx()
    ax2.set_ylabel('Davies-Bouldin Score', color='tab:orange')
    ax2.plot(cluster_range, davies_bouldin_scores, 's-', color='tab:orange', label='Davies-Bouldin Score')
    ax2.tick_params(axis='y', labelcolor='tab:orange')
    ax2.legend(loc='upper right')

    fig.tight_layout()
    plt.title('KMeans Evaluation Metrics')
    plt.show()

    return results

def evaluate_hierarchical_with_dendrogram(data, min_clusters=5, max_clusters=15):
    silhouette_scores = []
    davies_bouldin_scores = []
    cluster_range = range(min_clusters, max_clusters + 1)
    
    results = {}
    for n_clusters in cluster_range:
        hierarchical = AgglomerativeClustering(n_clusters=n_clusters)
        labels = hierarchical.fit_predict(data)
        
        silhouette_avg = silhouette_score(data, labels)
        davies_bouldin_avg = davies_bouldin_score(data, labels)
        
        silhouette_scores.append(silhouette_avg)
        davies_bouldin_scores.append(davies_bouldin_avg)
        
        results[n_clusters] = [hierarchical, silhouette_avg, davies_bouldin_avg]

    
    # Plotting the scores
    fig, ax1 = plt.subplots()

    ax1.set_xlabel('Number of clusters')
    ax1.set_ylabel('Silhouette Score', color='tab:blue')
    ax1.plot(cluster_range, silhouette_scores, 'o-', color='tab:blue', label='Silhouette Score')
    ax1.tick_params(axis='y', labelcolor='tab:blue')
    ax1.legend(loc='upper left')

    ax2 = ax1.twinx()
    ax2.set_ylabel('Davies-Bouldin Score', color='tab:orange')
    ax2.plot(cluster_range, davies_bouldin_scores, 's-', color='tab:orange', label='Davies-Bouldin Score')
    ax2.tick_params(axis='y', labelcolor='tab:orange')
    ax2.legend(loc='upper right')

    fig.tight_layout()
    plt.title('Hierarchical Clustering Evaluation Metrics')
    plt.show()
    
    # Plotting the dendrogram
    linked = linkage(data, 'ward')
    plt.figure(figsize=(10, 7))
    dendrogram(linked, truncate_mode='level', p=5)
    plt.title('Hierarchical Clustering Dendrogram')
    plt.xlabel('Data points')
    plt.ylabel('Distance')
    plt.show()

    return results

def plot_appliance_clusters(appliances, labels, n_clusters):
    plt.figure(figsize=(12, 6))
    for cluster_idx in range(n_clusters):
        if hasattr(appliances, 'loc'):
            cluster_data = appliances.loc[labels == cluster_idx]
        else:
            cluster_data = appliances[labels == cluster_idx]
            
        std_dev = np.std(cluster_data)
        
        plt.hist(cluster_data, bins=20, alpha=0.5, label=f'Cluster {cluster_idx}')
        
        mean = np.mean(cluster_data)
        
        plt.axvline(mean-std_dev, color=f'C{cluster_idx}', linestyle='dashed', linewidth=1.5, label=f'Std dev: {std_dev:.2f}')
        plt.axvline(mean+std_dev, color=f'C{cluster_idx}', linestyle='dashed', linewidth=1.5, label=f'Std dev: {std_dev:.2f}')
    
    plt.xlabel(f'Feature Appliances')
    plt.title(f'Cluster Distribution of Feature Appliances')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
def plot_cluster_distributions(X, clusterer, y=None):
    if hasattr(clusterer,'predict'):
        if y is None: labels = clusterer.predict(X)
        n_clusters = len(clusterer.cluster_centers_)
    else:
        if y is None: labels = clusterer.fit_predict(X)
        n_clusters = clusterer.n_clusters_
    if y is not None:
        labels=y
        
    num_features = X.shape[1]
    
    for feature_idx in range(num_features):
        plt.figure(figsize=(12, 6))
        for cluster_idx in range(n_clusters):
            if hasattr(X, 'loc'):
                cluster_data = X.loc[labels == cluster_idx, X.columns[feature_idx]]
            else:
                cluster_data = X[labels == cluster_idx]
                
            std_dev = np.std(cluster_data)
            
            plt.hist(cluster_data, bins=20, alpha=0.5, label=f'Cluster {cluster_idx}')
            
            mean = np.mean(cluster_data)
            
            plt.axvline(mean-std_dev, color=f'C{cluster_idx}', linestyle='dashed', linewidth=1.5, label=f'Std dev: {std_dev:.2f}')
            plt.axvline(mean+std_dev, color=f'C{cluster_idx}', linestyle='dashed', linewidth=1.5, label=f'Std dev: {std_dev:.2f}')
        
        
        plt.xlabel(f'Feature {X.columns[feature_idx]}')
        plt.title(f'Cluster Distribution of Feature {X.columns[feature_idx]}')
        plt.ylabel('Frequency')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

def plot_single_cluster_distribution(X, clusterer, cluster_idx, y=None):
    if hasattr(clusterer,'predict'):
        if y is None: labels = clusterer.predict(X)
        n_clusters = len(clusterer.cluster_centers_)
    else:
        if y is None: labels = clusterer.fit_predict(X)
        n_clusters = clusterer.n_clusters_
    if y is not None:
        labels=y
        
    num_features = X.shape[1]
    
    for feature_idx in range(num_features):
        plt.figure(figsize=(12, 6))

        if hasattr(X, 'loc'):
            cluster_data = X.loc[labels == cluster_idx, X.columns[feature_idx]]
        else:
            cluster_data = X[labels == cluster_idx]
            
        std_dev = np.std(cluster_data)
        
        plt.hist(cluster_data, bins=20, alpha=0.5, label=f'Cluster {cluster_idx}')
        
        mean = np.mean(cluster_data)
        
        plt.axvline(mean-std_dev, color=f'C{cluster_idx}', linestyle='dashed', linewidth=1.5, label=f'Std dev: {std_dev:.2f}')
        plt.axvline(mean+std_dev, color=f'C{cluster_idx}', linestyle='dashed', linewidth=1.5, label=f'Std dev: {std_dev:.2f}')
        
        
        plt.xlabel(f'Feature {X.columns[feature_idx]}')
        plt.title(f'Cluster Distribution of Feature {X.columns[feature_idx]}')
        plt.ylabel('Frequency')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()
    

In [ ]:
clusters = 5

In [ ]:
results = {}

# KMeans

## KMeans - Bez transformacija

In [ ]:
results['KMeans_NoTransf'] = evaluate_kmeans(train)
key = results['KMeans_NoTransf'][clusters][0]
visualize_clusters(train, key, perplexity = 50)
plot_cluster_distributions(train, key, key.predict(train))

## KMeans - Sa skaliranjem

In [ ]:
transf_map_scale = {}
data = scale(train, transf_map_scale)
results['KMeans_Scale'] = evaluate_kmeans(data)
key = results['KMeans_Scale'][clusters][0]
visualize_clusters(data, key, perplexity = 50)
plot_cluster_distributions(train, key, key.predict(data))

## KMeans - Sa skaliranjem i normalizacijom

In [ ]:
transf_map_norm = {}
data = scale(transform_skewed_attributes(train,transf_map_norm,0.1),transf_map_norm)
results['KMeans_ScaleNNorm'] = evaluate_kmeans(data)
key = results['KMeans_ScaleNNorm'][clusters][0]
visualize_clusters(data, key, perplexity = 50)
plot_cluster_distributions(train, key, key.predict(data))

## KMeans - Scale+Norm+PCA

In [ ]:
transf_map_pca_norm={}
pca = PCA(n_components=0.97)
data = scale(transform_skewed_attributes(train,transf_map_pca_norm,0.1),transf_map_pca_norm)
transf_data = pca.fit_transform(data)
results['KMeans_ScaleNormPCA'] = evaluate_kmeans(transf_data)
key = results['KMeans_ScaleNormPCA'][clusters][0]
visualize_clusters(transf_data, key, perplexity = 50)
plot_cluster_distributions(train, key, key.predict(transf_data))

## KMeans - Skaliranje+PCA

In [ ]:
clusters = 5
nesto = {}
pca_scale = PCA(n_components=0.97)
data = scale(train, nesto)
transf_data_scale = pca_scale.fit_transform(data)
results['KMeans_ScalePCA'] = evaluate_kmeans(transf_data_scale)
key = results['KMeans_ScalePCA'][clusters][0]
visualize_clusters(transf_data_scale, key, perplexity = 50)
plot_cluster_distributions(train, key, key.predict(transf_data_scale))

## KMeans - PCA

In [ ]:
pca_notrans = PCA(n_components=0.97)
data = pca_notrans.fit_transform(train)
results['KMeans_PCA'] = evaluate_kmeans(data)
key = results['KMeans_PCA'][clusters][0]
visualize_clusters(data, key, perplexity = 50)
plot_cluster_distributions(train, key, key.predict(data))

# Agglomerative

## Agglomerative clustering - Bez transformacija

In [ ]:
results['Agglomerative_NoTransf'] = evaluate_hierarchical_with_dendrogram(train)
key = results['Agglomerative_NoTransf'][clusters][0]
visualize_clusters(train, key, perplexity = 50)
plot_cluster_distributions(train, key, key.fit_predict(train))

## Agglomerative - Sa skaliranjem

In [ ]:
transf_map_scale = {}
data = scale(train, transf_map_scale)
results['Agglomerative_Scale'] = evaluate_hierarchical_with_dendrogram(data)
key = results['Agglomerative_Scale'][clusters][0]
visualize_clusters(data, key, perplexity = 50)
plot_cluster_distributions(train, key, key.fit_predict(data))

## Agglomerative - Sa skaliranjem i normalizacijom

In [ ]:
transf_map_norm = {}
data = scale(transform_skewed_attributes(train,transf_map_norm,0.1),transf_map_norm)
results['Agglomerative_ScaleNNorm'] = evaluate_hierarchical_with_dendrogram(data)
key = results['Agglomerative_ScaleNNorm'][clusters][0]
visualize_clusters(data, key, perplexity = 50)
plot_cluster_distributions(train, key, key.fit_predict(data))

## Agglomerative - Scale+Norm+PCA

In [ ]:
transf_map_pca_norm={}
pca = PCA(n_components=0.97)
data = scale(transform_skewed_attributes(train,transf_map_pca_norm,0.1),transf_map_pca_norm)
transf_data = pca.fit_transform(data)
results['Agglomerative_ScaleNormPCA'] = evaluate_hierarchical_with_dendrogram(transf_data)
key = results['Agglomerative_ScaleNormPCA'][clusters][0]
visualize_clusters(transf_data, key, perplexity = 50)
plot_cluster_distributions(train, key, key.fit_predict(transf_data))

## Agglomerative - Skaliranje+PCA

In [ ]:
nesto = {}
pca_scale = PCA(n_components=0.97)
data = scale(train,nesto)
transf_data_scale = pca_scale.fit_transform(data)
results['Agglomerative_ScalePCA'] = evaluate_hierarchical_with_dendrogram(transf_data_scale)
key = results['Agglomerative_ScalePCA'][clusters][0]
visualize_clusters(transf_data_scale, key, perplexity = 50)
plot_cluster_distributions(train, key, key.fit_predict(transf_data_scale))

## Agglomerative - PCA

In [ ]:
clusters = 7
pca_notrans = PCA(n_components=0.97)
data = pca_notrans.fit_transform(train)
results['Agglomerative_PCA'] = evaluate_hierarchical_with_dendrogram(data)
key = results['Agglomerative_PCA'][clusters][0]
visualize_clusters(data, key, perplexity = 50)
plot_cluster_distributions(train, key, key.fit_predict(data))

## Agglomarative + NonLinear DimRed

In [ ]:
clusters = 5
nesto = {}
iso = Isomap(n_components=3)
data = iso.fit_transform(train)
results['Agglomerative_IsoMap'] = evaluate_hierarchical_with_dendrogram(data)
key = results['Agglomerative_IsoMap'][clusters][0]
visualize_clusters(data, key, perplexity = 30)
plot_cluster_distributions(train, key, key.fit_predict(data))

# Rezultati

In [ ]:
def get_result_df(results:dict):
    res = {}
    for x in results.keys():
        res[x] = {}
        for y in results[x]:
            res[x][y] = results[x][y][1:]
    return pd.DataFrame(res)

def draw_results(df):
    all_values = np.array([value for pair in df.values.flatten() for value in pair])
    norm = plt.Normalize(all_values.min(), all_values.max())

    # Create the plot
    fig, ax = plt.subplots(figsize=(10, 5))

    # Iterate through the DataFrame and plot each cell as two blocks
    for i, row in enumerate(df.itertuples(index=False)):
        for j, (silh_score, davis_bondie_score) in enumerate(row):
            # Create a rectangle for silh_score
            ax.add_patch(plt.Rectangle((j, i + 0.5), 1, 0.5, color=plt.cm.viridis(norm(silh_score))))
            # Create a rectangle for davis_bondie_score
            ax.add_patch(plt.Rectangle((j, i), 1, 0.5, color=plt.cm.viridis(norm(davis_bondie_score))))

    # Set the limits and aspect
    ax.set_xlim(0, len(df.columns))
    ax.set_ylim(0, len(df))
    ax.set_aspect('equal')

    # Set the tick labels
    ax.set_xticks(np.arange(len(df.columns)) + 0.5)
    ax.set_yticks(np.arange(len(df)) + 0.5)
    ax.set_xticklabels(df.columns)
    ax.set_yticklabels(df.index)

    # Display the color bar
    sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=norm)
    sm.set_array([])
    plt.colorbar(sm, ax=ax, orientation='vertical')

    # Remove the axes for better visual effect
    ax.invert_yaxis()  # To match the DataFrame display
    ax.axis('off')

    # Display the plot
    plt.show()

In [ ]:
df = get_result_df(results)
draw_results(df)
#Jos ga ofarbaj

In [ ]:
df

**Agglomerative Clustering sa primenjenim PCAom nad podacima daje najbolje rezultate**
- Zato vršimo analizu tog rezultata

# Interpretacija rezultata

## Cluster 0

In [ ]:
plot_single_cluster_distribution(train, key, 0, key.fit_predict(data))

- Gender - cluster 0 ima približno jednak broj muškaraca i žena
- Age - cluster 0 uglavnom čine mlađe osobe (18-30 godina)
- Height - visina osoba u clusteru 0 teži nižim vrednostima (1.6-1.7m)
- Weight - najčešće kilaže su 50-70kg
- family_history_with_overweight - više od pola osoba u clusteru ima problema sa kilažom u porodici
- FAVC - u clusteru 0 osobe se često hrane kaloričnom hranom
- FCVC - u clusteru 0 osobe se relativno često hrane povrćem
- NCP - broj obroka je najčešće 3
- SMOKE - većina osoba u ovom clusteru ne puši
- CH2O - unos vode je umeren
- SCC - većina osoba u ovom clusteru ne vodi računa o unosu kalorija
- FAF - učestalost fizičke aktivnosti je manja do umerena, retke su osobe sa čestom fizičkom aktivnošću
- TUE - učestalost korišćenja tehnologije je manja do srednja
- NObeyesdad - večina osoba u ovoj kategoriji je underweight-normalna (0,1) ili vrlo gojazna (5-6)
- CAEC - većina osoba ovog clustera ponekad unose hranu između glavnih obroka
- CALC - većina osoba ovog clustera ponekad/nikad piju alkohol
- MTRANS - većina osoba ovog clustera koristi javni prevoz dok neki koriste i automobile, dok retko idu biciklom ili pešice

**Možemo da zaključimo da cluster čine ljudi niži ljudi, kilaže od 40-70kg, zbog čega nivoa gojaznosti varira između normalne težine i gojaznosti. Ostale karakteristike su prosečne (unos alkohola, tehnologije, transporta, unos vode i drugih atributa)**

## Cluster 1

In [ ]:
plot_single_cluster_distribution(train, key, 1, key.fit_predict(data))

- Gender - cluster 1 ima mnogo više žena nego muškaraca
- Age - cluster 1 uglavnom čine mlađe osobe 18-25 godina
- Height - visina osoba u clusteru 1 teži relativno visokim vrednostima 1.7-1.8m
- Weight - kilaža u ovom clusteru je dosta velika (minimum je oko 130kg)
- family_history_with_overweight - svi ljudi u ovoj grupi imaju problema sa kilažom u porodici
- FAVC - u clusteru 1 sve osobe se vrlo često hrane kaloričnom hranom
- FCVC - u clusteru 1 osobe se često hrane povrćem
- NCP - broj obroka je uvek 3
- SMOKE - nijedna osoba u ovom clusteru ne puši
- CH2O - unos vode je relativno visok
- SCC - većina osoba u ovom clusteru ne vodi računa o unosu kalorija
- FAF - učestalost fizičke aktivnosti je relativno mala-srednja
- TUE - učestalost korišćenja tehnologije je relativno mala
- NObeyesdad - sve osobe su gojazne (4)
- CAEC - većina osoba ovog clustera ponekad unose hranu između glavnih obroka
- CALC - većina osoba ovog clustera ponekad piju alkohol
- MTRANS - sve osoba ovog clustera koristi javni prevoz

**Možemo da zaključimo da cluster čine mlade ženske osobe, relativno visoke sa velikom kilažom. Sve osobe se hrane kaloričnom hranom i imaju problema sa kilažom u porodici. Gojaznost je zastupljena u ovom clusteru**

## Cluster 2

In [ ]:
plot_single_cluster_distribution(train, key, 2, key.fit_predict(data))

- Gender - cluster 2 ima mnogo više muškaraca nego žena
- Age - cluster 2 uglavnom čine mladi odrasli ljudi - 20-30 godina najćešće 
- Height - visina osoba u clusteru 2 se kreće od 1.6 do 1.8m, dok visine iznad 1.85 su ređe
- Weight - kilaža u ovom clusteru je dosta velika (minimum je oko 110kg)
- family_history_with_overweight - svi ljudi u ovoj grupi imaju problema sa kilažom u porodici
- FAVC - u clusteru 2 sve osobe se vrlo često hrane kaloričnom hranom
- FCVC - u clusteru 2 osobe se često hrane povrćem
- NCP - broj obroka je skoro uvek 3
- SMOKE - retki su pušači u ovom clusteru
- CH2O - unos vode je prosečan
- SCC - velika većina osoba u ovom clusteru ne vodi računa o unosu kalorija
- FAF - učestalost fizičke aktivnosti je relativno mala
- TUE - učestalost korišćenja tehnologije je relativno mala
- NObeyesdad - sve osobe su gojazne (3-4)
- CAEC - većina osoba ovog clustera ponekad unose hranu između glavnih obroka
- CALC - većina osoba ovog clustera ponekad piju alkohol
- MTRANS - u ovom clusteru osobe obično koriste javni prevoz ili automobil

**Možemo da zaključimo da cluster uglavnom čine odrasle muške osobe, prosečne visine sa većom kilažom. Sve osobe se hrane kaloričnom hranom i imaju problema sa kilažom u porodici. Pored toga retko se bave fizičkom aktivnošću i kreću se najčešće javnim prevozom ili automobilom. Gojaznost je zastupljena u ovom clusteru**

## Cluster 3

In [ ]:
plot_single_cluster_distribution(train, key, 3, key.fit_predict(data))

- Gender - cluster 3 ima približno jednak broj muškaraca i žena
- Age - cluster 3 uglavnom čine odrasli ljudi srednjih godina - 30-45 godina najćešće 
- Height - visina osoba u clusteru 3 se kreće od 1.5 do 1.85m
- Weight - kilaža u ovom clusteru je blago viša za datu visinu - 75-95kg
- family_history_with_overweight - većina ljudi u ovoj grupi imaju problema sa kilažom u porodici
- FAVC - u clusteru 3 većina osobe se vrlo često hrane kaloričnom hranom
- FCVC - u clusteru 3 osobe se prosečno hrane povrćem
- NCP - broj obroka je najčešće 3 ali ima i onih koji samo 1 obrok unose
- SMOKE - retki su pušači u ovom clusteru
- CH2O - svi nivoi unosa vode su prisutni
- SCC - velika većina osoba u ovom clusteru ne vodi računa o unosu kalorija
- FAF - učestalost fizičke aktivnosti je relativno mala
- TUE - učestalost korišćenja tehnologije je relativno mala
- NObeyesdad - osobe u ovom clusteru su najčešće blago gojazne (2) ili ekstremno gojazne
- CAEC - većina osoba ovog clustera ponekad unose hranu između glavnih obroka
- CALC - većina osoba ovog clustera ponekad piju alkohol ali ima puno njih koji nikada ne piju
- MTRANS - u ovom clusteru osobe obično voze automobil

**Možemo da zaključimo da cluster uglavnom čine odrasle osobe srednjih godina, niske-prosečne visine sa blago većom kilažom. Sve osobe se hrane kaloričnom hranom i imaju problema sa kilažom u porodici. Pored toga retko se bave fizičkom aktivnošću i kreću se najčešće automobilom. U okviru clustera je prisutna blaga do ozbiljna gojaznost**

## Cluster 4

In [ ]:
plot_single_cluster_distribution(train, key, 4, key.fit_predict(data))

- Gender - cluster 4 ima dosta više žena nego muškaraca
- Age - cluster 4 uglavnom čine mladi ljudi - 16-26 godina
- Height - visina osoba u clusteru 3 se kreće od 1.45 do 1.85 metara, pri čemu je 1.55-1.7m najčešća visina
- Weight - kilaža u ovom clusteru je dosta mala - 40-54kg
- family_history_with_overweight - većina ljudi u ovoj grupi nemaju problema sa kilažom u porodici
- FAVC - u clusteru 4 većina osobe se vrlo često hrane kaloričnom hranom
- FCVC - u clusteru 4 osobe se često hrane povrćem
- NCP - broj obroka je najčešće 3 ali ima i onih koji samo 1 obrok unose
- SMOKE - ekstremno su retki pušači u ovom clusteru
- CH2O - unos vode je nizak-srednji
- SCC - velika većina osoba u ovom clusteru ne vodi računa o unosu kalorija
- FAF - učestalost fizičke aktivnosti je relativno mala
- TUE - učestalost korišćenja tehnologije je relativno visoka
- NObeyesdad - osobe u ovom clusteru su najčešće neuhranjene ili normalne težine
- CAEC - većina osoba ovog clustera ponekad-često unose hranu između glavnih obroka
- CALC - većina osoba ovog clustera ponekad piju alkohol ali ima puno njih koji nikada ne piju
- MTRANS - u ovom clusteru osobe obično koriste javni prevoz

**Možemo da zaključimo da cluster uglavnom čine mlade žene, niske-prosečne visine sa malom kilažom. Većina osoba se hrani kaloričnom hranom ali nemaju problema sa kilažom u porodici. Pored toga retko se bave fizičkom aktivnošću, slabo unose vodu, često koriste tehnologiju. U okviru clustera je prisutna neuhranjenost i normalna kilaža**

## Cluster 5

In [ ]:
plot_single_cluster_distribution(train, key, 5, key.fit_predict(data))

- Gender - cluster 5 ima dosta više muškaraca nego žena
- Age - cluster 5 uglavnom čine mladi i srednjovečni ljudi - 20-40 godina
- Height - visina osoba u clusteru 5 se kreće od 1.55 do 1.90 metara, pri čemu je 1.6-1.8m najčešća visina
- Weight - kilaža u ovom clusteru je visoka za dati rang visina - 95-115kg
- family_history_with_overweight - većina ljudi u ovoj grupi imaju problema sa kilažom u porodici
- FAVC - u clusteru 5 velika većina osoba se vrlo često hrane kaloričnom hranom
- FCVC - u clusteru 5 osobe se dosta često hrane povrćem
- NCP - broj obroka je najčešće 3
- SMOKE - ekstremno su retki pušači u ovom clusteru
- CH2O - unos vode je zastupljen u svim opsezima
- SCC - velika većina osoba u ovom clusteru ne vodi računa o unosu kalorija
- FAF - učestalost fizičke aktivnosti je dosta mala
- TUE - učestalost korišćenja tehnologije je relativno visoka
- NObeyesdad - osobe u ovom clusteru imaju veću težinu od normalne, pri čemu je ozbiljna gojaznost retka
- CAEC - većina osoba ovog clustera ponekad unose hranu između glavnih obroka
- CALC - većina osoba ovog clustera ponekad piju alkohol ali ima puno njih koji nikada ne piju
- MTRANS - u ovom clusteru osobe obično koriste javni prevoz

**Možemo da zaključimo da cluster uglavnom čine mladi i srednjovečni muškarci, niske-prosečne visine sa većom kilažom. Većina osoba se hrani kaloričnom hranom ali i povrćem i imaju problema sa kilažom u porodici. Pored toga retko se bave fizičkom aktivnošću, često koriste tehnologiju. U okviru clustera je prisutna blago povišena kilaža kao i blaga gojaznost.**

## Cluster 6

In [ ]:
plot_single_cluster_distribution(train, key, 6, key.fit_predict(data))

- Gender - oba pola su zastupljena, ali muškaraca ima više
- Age - cluster 6 uglavnom čine mladi ljudi - 18-25 godina
- Height - visina osoba u clusteru 6 se kreće od 1.55 do 1.90 metara, pri čemu je 1.7-1.8m najčešća visina
- Weight - kilaža u ovom clusteru nije previsoka za dati raspon visina - 75-90kg
- family_history_with_overweight - većina ljudi u ovoj grupi imaju problema sa kilažom u porodici
- FAVC - u clusteru 6 velika većina osoba se vrlo često hrane kaloričnom hranom ali ima i onih koji retko se hrane kaloričnom hranom
- FCVC - u clusteru 6 osobe se prosečno često hrane povrćem
- NCP - broj obroka je najčešće 3 ali ima i puno onih koji imaju 1 obrok dnevno
- SMOKE - ekstremno su retki pušači u ovom clusteru
- CH2O - unos vode je zastupljen u svim opsezima, ali je prosečan unos najčešći
- SCC - velika većina osoba u ovom clusteru ne vodi računa o unosu kalorija
- FAF - učestalost fizičke aktivnosti varira, ima najviše onih koji se retko bave
- TUE - učestalost korišćenja tehnologije je relativno visoka
- NObeyesdad - puno osoba u ovom clusteru imaju velik stepen gojaznosti ali ima i onih sa normalnom težinom
- CAEC - većina osoba ovog clustera ponekad unose hranu između glavnih obroka
- CALC - većina osoba ovog clustera ponekad piju alkohol ali ima puno njih koji nikada ne piju
- MTRANS - u ovom clusteru osobe obično koriste javni prevoz ali i automobile

**Možemo da zaključimo da cluster uglavnom čine mladi ljudi, najčešće muškarci, prosečne do nadprosečne visine sa normalnom kilažom. Većina osoba se hrani kaloričnom hranom, donekle i povrćem i imaju problema sa kilažom u porodici. Pored toga retko-ponekad se bave fizičkom aktivnošću, često koriste tehnologiju. U okviru clustera je prisutna normalna kilaža kao i ekstremna gojaznost.**